In [7]:
import numpy as np
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial.distance import cosine

class centre_bow(object):
    def __init__(self, lim=250, thresh=0.3, thresh_sim=0.95, type_lim='word'):
        self.limit = lim
        self.topic_threshold = thresh
        self.sim_threshold = thresh_sim
        self.limit_type = type_lim


    def start_summary(self, sen_raw, sen_clean, len_lim):
        vectorizer = CountVectorizer(stop_words='english')
        word_mat_sent = vectorizer.fit_transform(sen_clean)

        my_transformer = TfidfTransformer(norm=None, sublinear_tf=False, smooth_idf=False)
        tf_idf = my_transformer.fit_transform(word_mat_sent)
        tf_idf = tf_idf.toarray()

        centrvec1 = tf_idf.sum(0)
        centrvec1 = np.divide(centrvec1, centrvec1.max())
        for i in range(centrvec1.shape[0]):
            if centrvec1[i] <= self.topic_threshold:
                centrvec1[i] = 0

        score_counts = []
        for i in range(tf_idf.shape[0]):
            score = self.get_similarity(tf_idf[i, :], centrvec1)
            score_counts.append((i, sen_raw[i], score, tf_idf[i, :]))

        sorted_scores = sorted(score_counts, key=lambda el: el[2], reverse=True)

        new_summaries = []

        for s in sorted_scores:
            if len(new_summaries) == len_lim:
                break
            include_flag = True
            for ps in new_summaries:
                sim = centre_bow().get_similarity(s[3], ps[3])

                if sim > centre_bow().sim_threshold:
                    include_flag = False
            if include_flag:
                new_summaries.append(s)
        
        results = []
        for item in new_summaries:
            results.append(item[1])
        return results

    @staticmethod
    def get_similarity(v1, v2):
        current_score = 0.0
        if np.count_nonzero(v1) != 0 and np.count_nonzero(v2) != 0:
            current_score = ((1 - cosine(v1, v2)) + 1) / 2
        return current_score